In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading the Data

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [19]:
PROJECT_PATH = '/content/drive/My Drive/Datasets/ML Project'
FIGURES_PATH = f'{PROJECT_PATH}/figures/preprocessing'
def get_figure_path(name: str):
  return f'{FIGURES_PATH}/{name}.png'
DATASETS_PATH = f'{PROJECT_PATH}/data/recommended'
TRAINING_PATH = f'{DATASETS_PATH}/training/training.csv'
TESTING_PATH = f'{DATASETS_PATH}/test/test.csv'
INDEX_COL = 'pkSeqID'
COLUMNS_ORDERED = [
    'min', 'max', 'mean', 'stddev',
    'saddr', 'sport', 'daddr', 'dport',
    'srate', 'drate',
    'N_IN_Conn_P_SrcIP', 'N_IN_Conn_P_DstIP',
    'state_number', 'proto',
    'seq',
    'attack', 'category', 'subcategory'
]

In [20]:
training = pd.read_csv(TRAINING_PATH, index_col=INDEX_COL)[COLUMNS_ORDERED]
training.head()

,min,max,mean,stddev,saddr,sport,daddr,dport,srate,drate,N_IN_Conn_P_SrcIP,N_IN_Conn_P_DstIP,state_number,proto,seq,attack,category,subcategory
pkSeqID,,,,,,,,,,,,,,,,,,
3142762,0.000000,4.031619,2.687519,1.900363,192.168.100.150,6551,192.168.100.3,80,0.494549,0.0,100,100,4,udp,251984,1,DDoS,UDP
2432264,3.856930,4.012924,3.934927,0.078003,192.168.100.150,5532,192.168.100.3,80,0.256493,0.0,38,100,3,tcp,256724,1,DDoS,TCP
1976315,2.974100,3.609205,3.341429,0.268666,192.168.100.147,27165,192.168.100.3,80,0.294880,0.0,100,100,3,tcp,62921,1,DDoS,TCP
1240757,0.000000,4.942302,3.222832,1.823185,192.168.100.150,48719,192.168.100.3,80,0.461435,0.0,63,63,4,udp,99168,1,DoS,UDP
3257991,2.979995,4.994452,3.983222,0.822418,192.168.100.147,22461,192.168.100.3,80,1.002999,0.0,100,100,4,udp,105063,1,DDoS,UDP


In [21]:
testing = pd.read_csv(TESTING_PATH, index_col=INDEX_COL)[COLUMNS_ORDERED]

In [29]:
def process_categories(cat):
  if str(cat).lower().startswith('theft'):
    return 'Theft'
  if str(cat).lower().startswith('normal'):
    return 'Normal'
  if cat == 'DoS HTTP':
    return 'DoS TCP'
  if cat == 'DDoS HTTP':
    return 'DDoS TCP'
  return cat


In [36]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()

In [39]:
X_train = training.drop(['attack' ,'category', 'subcategory'], axis=1)
y_train_cat = pd.DataFrame({'category': (training['category'] + ' ' + training['subcategory']).map(process_categories)})
y_train = cat_encoder.fit_transform(y_train_cat)

In [40]:
X_test = testing.drop(['attack', 'category', 'subcategory'], axis=1)
y_test_cat = pd.DataFrame({'category': (testing['category'] + ' ' + testing['subcategory']).map(process_categories)})
y_test = cat_encoder.transform(y_test_cat)

In [34]:
X_train.head()

,min,max,mean,stddev,saddr,sport,daddr,dport,srate,drate,N_IN_Conn_P_SrcIP,N_IN_Conn_P_DstIP,state_number,proto,seq
pkSeqID,,,,,,,,,,,,,,,
3142762,0.000000,4.031619,2.687519,1.900363,192.168.100.150,6551,192.168.100.3,80,0.494549,0.0,100,100,4,udp,251984
2432264,3.856930,4.012924,3.934927,0.078003,192.168.100.150,5532,192.168.100.3,80,0.256493,0.0,38,100,3,tcp,256724
1976315,2.974100,3.609205,3.341429,0.268666,192.168.100.147,27165,192.168.100.3,80,0.294880,0.0,100,100,3,tcp,62921
1240757,0.000000,4.942302,3.222832,1.823185,192.168.100.150,48719,192.168.100.3,80,0.461435,0.0,63,63,4,udp,99168
3257991,2.979995,4.994452,3.983222,0.822418,192.168.100.147,22461,192.168.100.3,80,1.002999,0.0,100,100,4,udp,105063


In [41]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

In [42]:
def process_port(i):
  return i

In [43]:
TO_DROP = ['saddr', 'daddr', 'seq']

transformer = ColumnTransformer([
  # ('drop', 'drop', TO_DROP),
  # ('encode', OneHotEncoder(), ['proto']),
  ('port', FunctionTransformer(func=process_port), ['sport', 'dport'])
], remainder='passthrough')
transformer.fit_transform(X_train)

         sport dport
pkSeqID             
3142762   6551    80
2432264   5532    80
1976315  27165    80
1240757  48719    80
3257991  22461    80
...        ...   ...
1132803  56044    80
3384621  21546    80
775893   30897    80
443484   36904    80
96906    14302    80

[2934817 rows x 2 columns]


array([['6551', '80', 0.0, ..., 4, 'udp', 251984],
       ['5532', '80', 3.85693, ..., 3, 'tcp', 256724],
       ['27165', '80', 2.9741, ..., 3, 'tcp', 62921],
       ...,
       ['30897', '80', 0.0, ..., 4, 'udp', 158616],
       ['36904', '80', 0.0, ..., 3, 'tcp', 179855],
       ['14302', '80', 0.0647669999999999, ..., 1, 'tcp', 95429]],
      dtype=object)